TRABALHO BD

A partir de webscrapping, ir buscar o número de ids escolhido (máx 20) de termos à escolha do utilizador e importar as informações para a base de dados

WEBSCRAPPING

In [47]:
import re

def validateInsertedText(item):
    return re.match(r'[\d|a-z|A-Z]+(\_?[\d|a-z|A-Z]+)*', item)

def url_get(item):
    url_list = []
    url = "https://www.ncbi.nlm.nih.gov/gene/?term={}".format(item)
    url_list.append(url)
    return url_list

query = input("Defina o termo que pretende pesquisar [bacteria/yeast/human...] \n Se o termo tiver mais do que uma palavra, separe por _: ")
try:
    
    if not validateInsertedText(query):
        raise Exception
    else:   
        print(url_get(query))

except Exception:
    print(f'Inválido. Insira um termo válido. O termo que inseriu foi: "{query}"')


['https://www.ncbi.nlm.nih.gov/gene/?term=malaria']


In [48]:
import requests, sys, json
content = []
for url in url_get(query):
    r = requests.get(url)
    content.append(r.content)
# print(content)

In [49]:
import requests
from bs4 import BeautifulSoup

# Parsing the HTML
for c in content:
    soup = BeautifulSoup(r.content, 'html.parser')
    res = soup.get_text()
# print(res)

In [50]:
import re
existe = re.findall(r'\s*ID:\s*\d*(?=\D)', res, re.DOTALL)
id_genes = ','.join(existe)
id_g = re.sub(r'\s*ID:\n*\s+', '', id_genes)
id_gene = id_g.split(',')
id_gene


['3043',
 '112314',
 '116906',
 '112313',
 '117578',
 '100101431',
 '100035820',
 '100034930',
 '493123',
 '493122',
 '100036393',
 '100036129',
 '100035705',
 '246749',
 '100036439',
 '100036484',
 '104057',
 '2539',
 '28',
 '118446222101']

In [51]:
try:
    number_of_genes = int(input("Número de genes a obter (máx: 20): "))
    print(number_of_genes)
    genes = id_gene[0:number_of_genes]
    print(genes)
except Exception:
    print("INVALID")

10
['3043', '112314', '116906', '112313', '117578', '100101431', '100035820', '100034930', '493123', '493122']


In [7]:
#BUSCAR LINKS DOS GENES

In [52]:
import time
def url_get_id(item):
    url_id = []
    for id in genes:
        url = f"https://www.ncbi.nlm.nih.gov/nuccore/{id}"
        url_id.append(url)
    return url_id
url_get_id(genes)

['https://www.ncbi.nlm.nih.gov/nuccore/3043',
 'https://www.ncbi.nlm.nih.gov/nuccore/112314',
 'https://www.ncbi.nlm.nih.gov/nuccore/116906',
 'https://www.ncbi.nlm.nih.gov/nuccore/112313',
 'https://www.ncbi.nlm.nih.gov/nuccore/117578',
 'https://www.ncbi.nlm.nih.gov/nuccore/100101431',
 'https://www.ncbi.nlm.nih.gov/nuccore/100035820',
 'https://www.ncbi.nlm.nih.gov/nuccore/100034930',
 'https://www.ncbi.nlm.nih.gov/nuccore/493123',
 'https://www.ncbi.nlm.nih.gov/nuccore/493122']

In [53]:
import requests
id_content = []
for url in url_get_id(genes):
    r = requests.get(url)
    id_content.append(r.content)
# id_content

In [54]:
import requests
from bs4 import BeautifulSoup

# Making a GET request
#r = requests.get('https://www.ncbi.nlm.nih.gov/nuccore/{}'.format(genes))
# Parsing the HTML
locus = ""
locus_list = []
for c in id_content:
	soup = BeautifulSoup(c, 'html.parser')

# Procurar um tag meta com um determinado atributo

	lines = soup.find_all('meta', {'name':"ncbi_uidlist"} )

	id = ""
	url = ""
	for line in lines:
		# print(line)
		# if 'name' in line.attrs:
		# 	print(line.attrs['name'])
		if 'content' in line.attrs:
			# print(line.attrs['content'])		
			id = line.attrs['content']

	if id:
		url ="https://www.ncbi.nlm.nih.gov/sviewer/viewer.fcgi?id={}&db=nuccore&report=genbank&conwithfeat=on&hide-cdd=on&retmode=text&ncbi_phid=CE88F25338A215A1000000000483042A&withmarkup=on&tool=portal&log$=seqview&maxdownloadsize=1000000".format(id)

	r2 = requests.get(url)
	locus += r2.content.decode('utf-8')

	if "Error" not in r2.content.decode('utf-8'):
		locus_list.append(r2.content.decode('utf-8'))
print(locus)
	#utf-8 para converter de bytes para string
	#https://stackoverflow.com/questions/31019854/typeerror-cant-use-a-string-pattern-on-a-bytes-like-object-in-re-findall

LOCUS       X56237                  1077 bp    mRNA    linear   PLN 12-JUN-2006
DEFINITION  N. crassa NUO-32 mRNA for NADH dehydrogenase 32 kD subunit.
ACCESSION   X56237
VERSION     X56237.1
KEYWORDS    NADH dehydrogenase (ubiquinone); NADH dehydrogenase subunit; NUO-32
            gene.
SOURCE      Neurospora crassa
  ORGANISM  Neurospora crassa
            Eukaryota; Fungi; Dikarya; Ascomycota; Pezizomycotina;
            Sordariomycetes; Sordariomycetidae; Sordariales; Sordariaceae;
            Neurospora.
REFERENCE   1  (bases 1 to 1077)
  AUTHORS   Van der Pas,J.C., Rohlen,D.A., Weidner,U. and Weiss,H.
  TITLE     Primary structure of the nuclear-encoded 29.9 kDa subunit of NADH:
            ubiquinone reductase from Neurospora crassa mitochondria
  JOURNAL   Biochim. Biophys. Acta 1089 (3), 389-390 (1991)
   PUBMED   1830489
REFERENCE   2  (bases 1 to 1077)
  AUTHORS   Roelen,D.A.
  TITLE     Direct Submission
  JOURNAL   Submitted (25-NOV-1990) Roehlen D.A., Institut fuer Bioch

LINK NCBI

In [55]:
import re
id_lista = []
# print(genes)
existe = re.findall(r'ACCESSION\s+[^\s]+', locus)
#\s+ um ou mais espaços
#[^\s]+ indica um conjunto de caracteres que se podem repetir uma ou mais vezes

if existe:
    for c in existe:
        m = re.match(r'ACCESSION\s+([^\s]+)', c )
        if m:
            id = m.group(1)
            id_lista.append("https://www.ncbi.nlm.nih.gov/nuccore/{}".format( id ) )
else:
    print( "Padrão não encontrado" )
id_lista


['https://www.ncbi.nlm.nih.gov/nuccore/X56237',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAADK0077081',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAADK0017277',
 'https://www.ncbi.nlm.nih.gov/nuccore/AAADK0016387',
 'https://www.ncbi.nlm.nih.gov/nuccore/L33068',
 'https://www.ncbi.nlm.nih.gov/nuccore/L33067']

ACCESSION

In [56]:
acc_list = []

existe = re.findall(r'ACCESSION\s+.*?(?=VERSION)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for accession in existe:
        m = re.match( r'ACCESSION\s+(.+)', accession, re.DOTALL )
        acc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
acc_list

['X56237 ',
 'AAADK0077081 ',
 'AAADK0017277 ',
 'AAADK0016387 ',
 'L33068 ',
 'L33067 ']

VERSION

In [57]:
version_list = []

existe = re.findall(r'VERSION.*?(?=KEYWORDS|DBLINK)', locus, re.DOTALL)
#\s+ um ou mais espaços
#\.* encontra o caractere ponto final zero ou mais vezes
#?= something que faz match com VERSION mas não consome estes carateres
if existe:
    for vers in existe:
        m = re.match( r'VERSION\s+(.+)', vers, re.DOTALL )
        version_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
#(.+) agrupar um ou mais caracteres dentro de um grupo
#re.sub(r'\s+', ' ' ...) substitui um ou mais espaços por apenas um espaço
version_list

['X56237.1 ',
 'AAADK0077081.1 ',
 'AAADK0017277.1 ',
 'AAADK0016387.1 ',
 'L33068.1 ',
 'L33067.1 ']

DEFINITION

In [58]:
defi_list = []
existe = re.findall(r'DEFINITION\s+.*?(?=ACCESSION)', locus, re.DOTALL)
if existe:
    for definition in existe:
        m = re.match( r'DEFINITION\s+(.+)', definition, re.DOTALL )
        defi_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
defi_list

['N. crassa NUO-32 mRNA for NADH dehydrogenase 32 kD subunit. ',
 'Mus musculus strain C57BL/6Jx129/Sv. ',
 'Mus musculus strain C57BL/6Jx129/Sv. ',
 'Mus musculus strain C57BL/6Jx129/Sv. ',
 'Human immunodeficiency virus type 1 proviral envelope glycoprotein (env) gene V4/CD4-binding region, partial sequence (LH20-2). ',
 'Human immunodeficiency virus type 1 proviral envelope glycoprotein (env) gene V4/CD4-binding region, partial sequence (LH20-1). ']

LOCUS

In [59]:
loc_list = []
existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
if existe:
    for loc in existe:
        m = re.match( r'LOCUS\s+(.+)', loc, re.DOTALL )
        loc_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_list

['X56237 1077 bp mRNA linear PLN 12-JUN-2006 ',
 'AAADK0077081 20 bp RNA linear ROD 20-SEP-2005 ',
 'AAADK0017277 21 bp RNA linear ROD 20-SEP-2005 ',
 'AAADK0016387 21 bp RNA linear ROD 20-SEP-2005 ',
 'HIV1LH22M 184 bp DNA linear VRL 31-JUL-1995 ',
 'HIV1LH21M 193 bp DNA linear VRL 31-JUL-1995 ']

LOCUS NAME

In [60]:
loc_name_list = []
existe = re.findall(r'LOCUS\s + [^\s]+', locus, re.DOTALL)
if existe:
    for id in existe:
        m = re.match( r'LOCUS\s+(.+)', id, re.DOTALL )
        loc_name_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
loc_name_list


['X56237',
 'AAADK0077081',
 'AAADK0017277',
 'AAADK0016387',
 'HIV1LH22M',
 'HIV1LH21M']

NUMBER OF BP

In [61]:
loc_bp_list = []
loc = []

existe = re.findall(r'LOCUS\s+.*?(?=DEFINITION)', locus, re.DOTALL)
# print(existe)
if existe:
    for c in existe:
        loc.append(list(filter(None, c.split(' '))))
for x in range(len(loc)):
    loc_bp_list.append(' '.join(loc[x][2:4])) 


loc_bp_list

['1077 bp', '20 bp', '21 bp', '21 bp', '184 bp', '193 bp']

MOLECULE TYPE

In [62]:
mol_type = []
for m in range(len(loc)):
    if "linear" in loc[m][5]:
        mol_type.append(' '.join(loc[m][4:6]))
    else:
        mol_type.append(' '.join(loc[m][4:5]))

mol_type

['mRNA linear',
 'RNA linear',
 'RNA linear',
 'RNA linear',
 'DNA linear',
 'DNA linear']

GENBANK DIVISION

In [63]:
gb_div = []
for gb in range(len(loc)):
    gb_div.append(' '.join(loc[gb][6:7]))
gb_div

['PLN', 'ROD', 'ROD', 'ROD', 'VRL', 'VRL']

MODIFICATION DATE

In [64]:
mod_date = []
for md in range(len(loc)):
    mod_date.append(''.join(loc[md][7]).replace("\n",""))
mod_date

['12-JUN-2006',
 '20-SEP-2005',
 '20-SEP-2005',
 '20-SEP-2005',
 '31-JUL-1995',
 '31-JUL-1995']

SOURCE

In [65]:
source_list = []
existe = re.findall(r'SOURCE\s+.*?(?=ORGANISM)', locus, re.DOTALL)
# print(existe)
if existe:
    for source in existe:
        m = re.match( r'SOURCE\s+(.+)', source, re.DOTALL )
        source_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
source_list

['Neurospora crassa ',
 'Mus musculus (house mouse) ',
 'Mus musculus (house mouse) ',
 'Mus musculus (house mouse) ',
 'Human immunodeficiency virus 1 (HIV-1) ',
 'Human immunodeficiency virus 1 (HIV-1) ']

ORGANISM

In [66]:
org_list = []
existe = re.findall(r'ORGANISM\s+.*?(?=\n)', locus, re.DOTALL)
if existe:
    for org in existe:
        m = re.match( r'ORGANISM\s+(.+)', org, re.DOTALL )
        org_list.append(re.sub(r'\s+', ' ', m.group(1) ) )
org_list

['Neurospora crassa',
 'Mus musculus',
 'Mus musculus',
 'Mus musculus',
 'Human immunodeficiency virus 1',
 'Human immunodeficiency virus 1']

FEATURES - CDS

In [67]:
from Bio import SeqIO

In [68]:
from Bio import Entrez
from Bio import SeqIO
from Bio import SeqFeature
from Bio.SeqFeature import SeqFeature, FeatureLocation

Entrez.email = "pg49849@alunos.uminho.pt"

features=[]
CDS=[] 
source = []
gene = []


for acc in acc_list:
    handle = Entrez.efetch(db="nucleotide", rettype = "gb", retmode= "text", id=acc)
    
    for record in SeqIO.parse(handle,"gb"):
        
        for feature in record.features:
            features.append(feature.type)

        aux_source = []
        aux_gene = []


        for feature in record.features:
            if feature.type == "source":
                aux_source.append(str(feature.location))

            if feature.type == 'gene':
                aux_gene.append(feature.qualifiers["gene"])
 
        gene.append(aux_gene)
        source.append(aux_source)

        record_CDS = []
        for feature in record.features:
            CDS_aux=[]
            if feature.type == "CDS":
                CDS_aux.append(str(feature.location))
                CDS_aux.append(feature.qualifiers["product"])
                CDS_aux.append(feature.qualifiers["protein_id"])
                CDS_aux.append(feature.qualifiers["translation"])
                # print('->' , CDS_aux)
            record_CDS.append(CDS_aux)
            
        CDS.append(record_CDS)

# print(source)
# print(gene)
# print(exon)
# print(intron)
print(features)
print('*',source)
print('->',gene)
print(len(CDS))
print('p', CDS)  
print('->', CDS) 

['source', 'gene', 'mRNA', 'CDS', 'sig_peptide', 'mat_peptide', 'polyA_site', 'source', 'source', 'source', 'source', 'gene', 'CDS', 'source', 'gene', 'CDS']
* [['[0:1077](+)'], ['[0:20](+)'], ['[0:21](+)'], ['[0:21](+)'], ['[0:184](+)'], ['[0:193](+)']]
-> [[['NUO-32']], [], [], [], [['env']], [['env']]]
6
p [[[], [], [], ['[67:889](+)', ['NADH dehydrogenase subunit'], ['CAA39694.1'], ['MRAALRLLATATATVRPSARFLKPGSPTGLTGLGTHPSPRSALLYLYNHTLDKLKQIPEHSLYRQSAEALTKHRLAIVEQYVPDGYDAWQERARKLLEKHKSDLTARQFDGQHARLVEGPDGRAYFIRQMVPPQDWRDVEWDGAVLDPHFSWVQTGEDVVGAVKLEDSDKLLELDKIRESDPVAYRQGLRDLGIKMGGVVEDKSPVEWESEPPLSAEQIAEMEARIGSGLIEEVVQVAEGELKLVDIMTQARPWEALEEEAPEGQWTYFERKE']], [], [], []], [[]], [[]], [[]], [[], [], ['[<0:>184](+)', ['envelope glycoprotein'], ['AAA73551.1'], ['CNTSRLFNSTWNGTEGLPDITLPCRIKQIINMWQEVGKAMYAPPIKGQIRCSSNITGLLLT']]], [[], [], ['[<0:>193](+)', ['envelope glycoprotein'], ['AAA73550.1'], ['CNTSQLFNSTWNGTEGFPNTENITLPCRIKQIINLWQEVGKAMYAPPIRGQIRCSSNITGLLLT']]]]
-> [[[], [], [], ['[6

SEQUENCE/ORIGIN

In [69]:
seq_origin = []
existe = re.findall(r'ORIGIN\s+.*?(?=//)', locus, re.DOTALL)
if existe:
    for nuc in existe:
        m = re.match( r'ORIGIN\s+(.+)', nuc, re.DOTALL)
        a = re.sub(r'\s+', '', m.group(1) ) 
        num = re.sub(r'\d+','', a)
        seq_origin.append(num)
seq_origin

['ccacaatccaccccaccccaccggaccccgagtcagtcaacaacctccacgacacgactcggaaaccatgcgcgcggcactccgattgctcgcgaccgcgacggctaccgtccgcccgtccgcccgcttcctcaaacccggctccccgaccggcctcaccggcctcggcacccacccgtcgccgcgctccgccctgctgtacctctacaaccacaccctcgacaagctcaagcagatccccgagcactcgctgtaccgccagtccgccgaggccctgaccaagcaccgcctcgccatcgtcgagcagtacgtgcccgacggctacgacgcctggcaggagcgcgcccgcaagctgctcgagaagcacaagagcgacctcacggcccgccagttcgacggccagcatgcccgcctagtcgaaggccccgacggccgcgcttacttcatccgccagatggtccccccgcaggactggcgcgacgtcgagtgggatggtgccgtcctggatccgcacttttcctgggttcagaccggcgaggatgtcgtcggcgccgtcaagctggaagacagcgacaagctgctcgagctggacaagatcagggaatcagacccggttgcctatcgccagggtctcagggatctcggtatcaagatgggcggtgttgtcgaggacaagagccccgtcgagtgggagtcggagccgccgttgagcgctgaacagattgctgagatggaggccaggattggttctggcttgattgaggaggttgttcaggttgccgagggcgagcttaagctggttgacattatgacccaggccaggccttgggaagctcttgaggaggaggcgccagagggccaatggacttactttgagcgtaaggagtaaacaaacatcgacagacatagagaatgggaacacaggaccaaccaggccatgacggccacctattttatgagcgttacgaagaagaagggaggacaggcatttctcagcc

In [70]:
seq_count = []
for seq in seq_origin:
    #contagem de nucleotidos
    A= []
    C= []
    T= []
    G= []
    for n in seq:
        if n == 'a':
            A.append('a')
        if n == 'c':
            C.append('c')
        if n == 't':
            T.append('t')
        if n == 'g':
            G.append('g')
        
    a = len(A)
    c = len(C)
    t = len(T)
    g = len(G)
    seq_count.append([a,c,t,g])
    #fazer append de um array
seq_count


[[233, 353, 179, 312],
 [1, 7, 4, 8],
 [4, 4, 4, 9],
 [5, 8, 5, 3],
 [71, 36, 45, 32],
 [74, 38, 48, 33]]

In [71]:
length = []
for seq in seq_origin:
    length.append(len(seq))
length

[1077, 20, 21, 21, 184, 193]

PUBMED

In [72]:
pub_list = []
existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE|COMMENT)', locus, re.DOTALL)
# print(existe)
if existe:
    for pubm in existe:
        m = re.match( r'PUBMED\s+(.+)?(?=\w)', pubm, re.DOTALL )
        pub_list.append(re.sub(r'\s+\w+.*', ' ', m.group(1) ) )
print(pub_list)

#procurar os ficheiros que tem o campo PUBMED, se não tiver vai valor vazio na lista (para fazer a relação genbank-PM)
file_locus_pub_list = []
for x in locus_list:
    aux_pub_list = []
    existe = re.findall(r'PUBMED\s+.*?(?=REFERENCE|COMMENT)', x, re.DOTALL)
    # print(existe)
    if existe:
        for pubm in existe:
            m = re.match( r'PUBMED\s+(.+)?(?=\w)', pubm, re.DOTALL )

            aux_pub_list.append(re.sub(r'\s+\w+.*', ' ', m.group(1) ) )
    else:
        aux_pub_list.append('')
    file_locus_pub_list.append(aux_pub_list)

print(file_locus_pub_list)

['183048', '1466314', '1496030', '16141072 ', '1614107', '16645617 ', '1466314', '1496030', '16141072 ', '1614107', '16645617 ', '1466314', '1496030', '16141072 ', '1614107', '16645617 ', '787013', '787013']
[['183048'], ['1466314', '1496030', '16141072 ', '1614107', '16645617 '], ['1466314', '1496030', '16141072 ', '1614107', '16645617 '], ['1466314', '1496030', '16141072 ', '1614107', '16645617 '], ['787013'], ['787013']]


In [73]:
import requests
from bs4 import BeautifulSoup

def url_get_id_pm(item):
    url_id_pm = []
    for id_pm in item:
        url = f"https://pubmed.ncbi.nlm.nih.gov/{id_pm}/"
        url_id_pm.append(url)
    return url_id_pm

pm = ""
soup = []
content_pm = []
urls = url_get_id_pm(pub_list)
print(urls)

for url in urls:
	r2 = requests.get(url)
	pm += r2.content.decode('utf-8')
	soup.append(BeautifulSoup(r2.content, 'html.parser'))
	content_pm.append(r2.content.decode('utf-8'))

#print(soup)
#print(content_pm)




['https://pubmed.ncbi.nlm.nih.gov/183048/', 'https://pubmed.ncbi.nlm.nih.gov/1466314/', 'https://pubmed.ncbi.nlm.nih.gov/1496030/', 'https://pubmed.ncbi.nlm.nih.gov/16141072 /', 'https://pubmed.ncbi.nlm.nih.gov/1614107/', 'https://pubmed.ncbi.nlm.nih.gov/16645617 /', 'https://pubmed.ncbi.nlm.nih.gov/1466314/', 'https://pubmed.ncbi.nlm.nih.gov/1496030/', 'https://pubmed.ncbi.nlm.nih.gov/16141072 /', 'https://pubmed.ncbi.nlm.nih.gov/1614107/', 'https://pubmed.ncbi.nlm.nih.gov/16645617 /', 'https://pubmed.ncbi.nlm.nih.gov/1466314/', 'https://pubmed.ncbi.nlm.nih.gov/1496030/', 'https://pubmed.ncbi.nlm.nih.gov/16141072 /', 'https://pubmed.ncbi.nlm.nih.gov/1614107/', 'https://pubmed.ncbi.nlm.nih.gov/16645617 /', 'https://pubmed.ncbi.nlm.nih.gov/787013/', 'https://pubmed.ncbi.nlm.nih.gov/787013/']


AUTHOR, JOURNAL, TITLE, AFFILIATION

In [74]:
import re
from Bio import SeqIO
from Bio import Entrez
from Bio import Medline
import pandas as pd
import numpy as np

In [75]:
Ids=[]
title_list = []
abstract = []
author = []
affiliation = []
journal = []
database = 'PubMed'
Entrez.email= "pg49836@alunos.uminho.pt"
# idlist= pub_list
idlist = []
handle = Entrez.efetch(db=database, id=pub_list, rettype="medline", retmode="text") 
records = Medline.parse(handle)
description=[]
for info in records:
    title_list.append(info.get("TI", "-"))
    author.append(info.get("AU", "-"))
    journal.append(info.get("SO", "-"))
    affiliation.append(info.get("AD", "-") )
    abstract.append(info.get("AB", "-"))
    print()
print(title_list)
print(abstract)
print(author)
print(affiliation)
print(journal)









["[Short recapitulation for the next thirsty spell: our kidneys - the body's water police].", 'Therapeutic management of triglycerides: an international perspective.', '[The relation between venous stasis and the occurrence of pain].', 'The transcriptional landscape of the mammalian genome.', '[The fate of corneal transplants from the Zurich eye bank].', 'Genome-wide analysis of mammalian promoter architecture and evolution.', 'A comparison between conventional gingivectomy and a non-surgical regime in the treatment of periodontitis.']
['-', 'Current recommendations from various international expert committees generally concur in their definitions of borderline and high triglyceride levels, with small but important differences between recommendations in the definition of normal levels. However, population-based data on triglyceride levels are poorly developed in most countries, making difficult any international comparisons of prevalences of hypertriglyceridemia using the new de

CONNECTION TO DATABASE

In [76]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="Project_TJM"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`LOCUS` (
        `Id_locus` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Accession` VARCHAR(100) NOT NULL,
        `Sequence_length` INT NULL,
        `Molecule_type` VARCHAR(20) NULL,
        `GB_division` VARCHAR(10) NULL,
        `Source` VARCHAR(200) NULL,
        `Organism` VARCHAR(200) NULL,
        `Locus_name` VARCHAR(45) NULL
        );
    """
    Cursor.execute(TableName)
    sql_loc = "INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES"
    loc_name_list = list(dict.fromkeys(loc_name_list))  
    for index in range(len(loc_name_list)):
       
        select_ids_loc = f"SELECT Id_Locus FROM LOCUS WHERE Locus_name = \"{loc_name_list[index]}\""
        Cursor.execute(select_ids_loc)
        myresult_locus = Cursor.fetchall()
        id_loc = None
        for res_loc, in myresult_locus:
            id_loc = res_loc
        if id_loc is None: 
            if index == len(loc_name_list)-1:
                sql_loc += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\")"
            else:
                sql_loc += f" (\"{loc_name_list[index]}\", \"{length[index]}\", \"{mol_type[index]}\", \"{gb_div[index]}\", \"{source_list[index]}\", \"{org_list[index]}\", \"{acc_list[index]}\"),"
    print(sql_loc)
    if sql_loc != "INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES": 
        if sql_loc.endswith(","):  
            sql_loc = sql_loc[:-1]
        Cursor.execute(sql_loc)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO LOCUS (Locus_name, Sequence_length, Molecule_type, GB_division, Source, Organism, Accession) VALUES ("X56237", "1077", "mRNA linear", "PLN", "Neurospora crassa ", "Neurospora crassa", "X56237 "), ("AAADK0077081", "20", "RNA linear", "ROD", "Mus musculus (house mouse) ", "Mus musculus", "AAADK0077081 "), ("AAADK0017277", "21", "RNA linear", "ROD", "Mus musculus (house mouse) ", "Mus musculus", "AAADK0017277 "), ("AAADK0016387", "21", "RNA linear", "ROD", "Mus musculus (house mouse) ", "Mus musculus", "AAADK0016387 "), ("HIV1LH22M", "184", "DNA linear", "VRL", "Human immunodeficiency virus 1 (HIV-1) ", "Human immunodeficiency virus 1", "L33068 "), ("HIV1LH21M", "193", "DNA linear", "VRL", "Human immunodeficiency virus 1 (HIV-1) ", "Human immunodeficiency virus 1", "L33067 ")
6 record inserted.


In [77]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="Project_TJM"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`GENBANK` (
        `Id_GB` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `GB_id` VARCHAR(45) NOT NULL,
        `Definition` VARCHAR(400) NULL,
        `Modification_date` VARCHAR(11) NULL,
        `Version` VARCHAR(20) NOT NULL UNIQUE,
        `LOCUS_Id_locus` INT NOT NULL,
        `ORIGIN_Id_origin` INT NULL
    )
    """
    Cursor.execute(TableName)
    sql_gb = "INSERT INTO GENBANK (GB_Id, Modification_date, Definition, Version, LOCUS_Id_locus, ORIGIN_Id_origin) VALUES"
    

    for index in range(len(version_list)):
        sql_insert_origin = "INSERT INTO `Project_TJM`.ORIGIN (Sequence, Length, Adenine, Citosine, Timine , Guanine) VALUES"
        select_ids_origin = f"SELECT Id_Origin FROM ORIGIN WHERE Sequence = \"{seq_origin[index]}\""
        Cursor.execute(select_ids_origin)
        myresult_origin = Cursor.fetchall()
        id_ori = None
        for res_ori, in myresult_origin:
            id_ori = res_ori

        if id_ori is None: 
            if index == len(loc_name_list)-1:
                sql_insert_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
            else:
                sql_insert_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\"),"

        if sql_insert_origin != "INSERT INTO `Project_TJM`.ORIGIN (Sequence, Length, Adenine, Citosine, Timine , Guanine) VALUES":
            # select_ids_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
            if sql_insert_origin.endswith(","):  #remove ultimo caracter se este for uma ","
                sql_insert_origin = sql_insert_origin[:-1] #remove utimo caracter da string
            Cursor.execute(sql_insert_origin)

            # print(Cursor.lastrowid)  #buscar o id do origin inserido
            id_inserted_origin = Cursor.lastrowid
        else:
            id_inserted_origin = id_ori


        #fazer a verificação se existe locus repetido (evitar duplicados)
        select_locus = f"SELECT Id_locus from LOCUS where Locus_name = '{loc_name_list[index]}'"
        Cursor.execute(select_locus)
        myresult = Cursor.fetchall()
        id_locus = None
        for res, in myresult:
            id_locus = res
        
        if id_locus is not None:
            select_gb = f"SELECT Id_GB from GENBANK where GB_Id = '{loc_name_list[index]}'"
            Cursor.execute(select_gb)
            myresult_gb = Cursor.fetchall()
            id_genbank = None
            for res_gb, in myresult_gb:
                id_genbank = res_gb
            
            if id_genbank is None:
                if index == len(version_list)-1:
                    sql_gb += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\")"
                else:
                    sql_gb += f" (\"{loc_name_list[index]}\", \"{mod_date[index]}\", \"{defi_list[index]}\", \"{version_list[index]}\", \"{id_locus}\", \"{id_inserted_origin}\"),"
                
    if sql_gb != "INSERT INTO GENBANK (GB_Id, Modification_date, Definition, Version, LOCUS_Id_locus, ORIGIN_Id_origin) VALUES":

        if sql_gb.endswith(","):  #remove ultimo caracter se este for uma ","
            sql_gb = sql_gb[:-1] #remove utimo caracter da string
            
        Cursor.execute(sql_gb)

        DataBase.commit()

        print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

6 record inserted.


In [78]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="Project_TJM"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`PUBMED` (
        `Id_PM` INT NOT NULL AUTO_INCREMENT,
        `PM_ID` INT NOT NULL,
        `Journal` VARCHAR(100) NULL,
        `Title` VARCHAR(500) NULL,
        `Abstract` LONGTEXT NULL,
        PRIMARY KEY (`Id_PM`)
    )
    """
    Cursor.execute(TableName)
    
    #print(journal)
    #print(len(title_list))
    #print(len(abstract))
    #print(len(pub_list))
    sql_pm = "INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES"
    pub_list = list(dict.fromkeys(pub_list))  #tirar os duplicados (também n apanha o ultimo)
    #print(pub_list)
    for index in range(len(pub_list)):
        select_pm = f"SELECT Id_PM from PUBMED where PM_Id = '{pub_list[index]}'"
        Cursor.execute(select_pm)
        myresult_pubmed = Cursor.fetchall()
        id_pubmed = None
        for res_pm, in myresult_pubmed:
            id_pubmed = res_pm
        
        if id_pubmed is None:
            if index == len(pub_list)-1:
                sql_pm += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\")"
            else:
                sql_pm += f" (\"{pub_list[index]}\", \"{journal[index]}\", \"{title_list[index]}\", \"{abstract[index]}\"),"
        
    if sql_pm != "INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES":
        # select_ids_origin += f" (\"{seq_origin[index]}\", \"{length[index]}\", \"{seq_count[index][0]}\", \"{seq_count[index][1]}\", \"{seq_count[index][2]}\",  \"{seq_count[index][3]}\")"
        if sql_pm.endswith(","):  #remove ultimo caracter se este for uma ","
            sql_pm = sql_pm[:-1] #remove utimo caracter da string
        print(sql_pm)
        Cursor.execute(sql_pm)

        DataBase.commit()

        print(Cursor.rowcount, "record inserted.")

    Cursor.close()
    print(pub_list)

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO PUBMED (PM_ID, Journal, Title, Abstract) VALUES ("183048", "Krankenpflege (Frankf). 1976 Jun;30(6):193-4.", "[Short recapitulation for the next thirsty spell: our kidneys - the body's water police].", "-"), ("1466314", "Am J Cardiol. 1992 Dec 14;70(19):26H-31H. doi: 10.1016/0002-9149(92)91087-k.", "Therapeutic management of triglycerides: an international perspective.", "Current recommendations from various international expert committees generally concur in their definitions of borderline and high triglyceride levels, with small but important differences between recommendations in the definition of normal levels. However, population-based data on triglyceride levels are poorly developed in most countries, making difficult any international comparisons of prevalences of hypertriglyceridemia using the new definitions. However, it is probable that there should be considerable differences in the prevalence of hypertriglyceridemia, probably due to a mixture of genetic and envir

In [83]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="Project_TJM"
    )
    Cursor = DataBase.cursor()

    '''TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """ '''
    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`PM_GB_relationship` (
        `PUBMED_Id_PM` INT NOT NULL,
        `GENBANK_Id_GB` INT NOT NULL,
        PRIMARY KEY (`PUBMED_Id_PM`, `GENBANK_Id_GB`)
    )
    """
    Cursor.execute(TableName)
    print(len(file_locus_pub_list))
    print(len(loc_name_list))

    for index_ids in range(len(file_locus_pub_list)):
        '''for index in range(len(author[index_ids])):
            #verificar se existe autor (evitar duplicados)
            select_ids_auth = f"SELECT Id_Author FROM AUTHORS WHERE Name = \"{author[index_ids]}\""
            Cursor.execute(select_ids_auth)
            myresult_auth = Cursor.fetchall()
            id_auth = None
            for res, in myresult_auth:
                id_affiliation = res'''

        if file_locus_pub_list[index_ids][0] != '':
            db_id_pm = []
            for pm_ids in file_locus_pub_list[index_ids]:
                select_pm_gb = f"SELECT Id_PM FROM PUBMED WHERE PM_ID = \"{pm_ids}\""
                
                #print(select_pm_gb)
                Cursor.execute(select_pm_gb)
                myresult_pm = Cursor.fetchall()
                id_pm = None
                for res, in myresult_pm:
                    id_pm = res
                db_id_pm.append(id_pm)

            print(db_id_pm)
            select_gb_pm = f"SELECT Id_GB FROM GENBANK WHERE GB_ID= \"{loc_name_list[index_ids]}\""
            print(select_gb_pm)
            Cursor.execute(select_gb_pm)
            myresult_gb = Cursor.fetchall()
            id_gb = None
            for res, in myresult_gb:
                id_gb = res

            index = 0
            sql_pm_gb_relation = "INSERT INTO PM_GB_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES" 
            for id_pm in db_id_pm:
                if id_gb is not None and id_pm is not None:      
                    #verificar se existe relação entre ids
                    select_rel_gb_pm = f"SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM PM_GB_relationship WHERE PUBMED_Id_PM = \"{id_pm}\" and GENBANK_Id_GB = \"{id_gb}\""
                    print(select_rel_gb_pm)
                    Cursor.execute(select_rel_gb_pm)
                    myresult_rel_pm_gb = Cursor.fetchall()
                    id_rel_pm_gb = None
                    for res_PUBMED_Id_PM,res_GENBANK_Id_GB, in myresult_rel_pm_gb:   #apenas selecionamos um dos id pq se não houver relação serão ambos null
                        id_rel_pm_gb = res_PUBMED_Id_PM     

                    #se null:
                    if id_rel_pm_gb is None:
                        if index == len(db_id_pm)-1:
                            sql_pm_gb_relation += f" ({id_pm}, {id_gb})"
                        else:
                            sql_pm_gb_relation += f" ({id_pm}, {id_gb}),"
                index += 1

            print(sql_pm_gb_relation)
            if sql_pm_gb_relation != "INSERT INTO PM_GB_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES":
                #remove ultimo caracter se este for uma ","
                if sql_pm_gb_relation.endswith(","):
                    sql_pm_gb_relation = sql_pm_gb_relation[:-1] #remove utimo caracter da string

                print(sql_pm_gb_relation)
                Cursor.execute(sql_pm_gb_relation)
                DataBase.commit()
                print(Cursor.rowcount, "record inserted.")

    
    
    
    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

6
6
[2]
SELECT Id_GB FROM GENBANK WHERE GB_ID= "X56237"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM PM_GB_relationship WHERE PUBMED_Id_PM = "2" and GENBANK_Id_GB = "9"
INSERT INTO PM_GB_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (2, 9)
INSERT INTO PM_GB_relationship (PUBMED_Id_PM, GENBANK_Id_GB) VALUES (2, 9)
1 record inserted.
[3, 4, 5, 6, 7]
SELECT Id_GB FROM GENBANK WHERE GB_ID= "AAADK0077081"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM PM_GB_relationship WHERE PUBMED_Id_PM = "3" and GENBANK_Id_GB = "10"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM PM_GB_relationship WHERE PUBMED_Id_PM = "4" and GENBANK_Id_GB = "10"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM PM_GB_relationship WHERE PUBMED_Id_PM = "5" and GENBANK_Id_GB = "10"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM PM_GB_relationship WHERE PUBMED_Id_PM = "6" and GENBANK_Id_GB = "10"
SELECT PUBMED_Id_PM, GENBANK_Id_GB FROM PM_GB_relationship WHERE PUBMED_Id_PM = "7" and GENBANK_Id_GB = "10"
INSERT INTO PM_GB_relationship (PUBMED_Id_PM, GENBAN

In [80]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="Project_TJM"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`AUTHORS` (
        `Id_Author` INT NOT NULL AUTO_INCREMENT,
        `Name` VARCHAR(45) NULL,
        `AFFILIATION_Id_affiliation` INT NULL,
        PRIMARY KEY (`Id_Author`)
    )
    """
    Cursor.execute(TableName)

    for index_pubmed in range(len(pub_list)):
        
        affiliation_ids = []
        for index in range(len(affiliation[index_pubmed])):

            #verificar se existe afiliação (evitar duplicados)
            select_affiliation = f"SELECT Id_affiliation FROM `Project_TJM`.AFFILIATION WHERE Description = \"{affiliation[index_pubmed][index]}\""
            Cursor.execute(select_affiliation)
            myresult_af = Cursor.fetchall()
            id_affiliation = None
            for res, in myresult_af:
                id_affiliation = res


            if id_affiliation is not None:
                affiliation_ids.append(id_affiliation)
            else:
                sql_insert_affiliation = "INSERT INTO `Project_TJM`.AFFILIATION (Description) VALUES" 
                sql_insert_affiliation += f" (\"{affiliation[index_pubmed][index]}\")"
                print(sql_insert_affiliation)
                Cursor.execute(sql_insert_affiliation)
                # print(Cursor.lastrowid)  #buscar o id do affiliation inserido
                affiliation_ids.append(Cursor.lastrowid)


        sql = "INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES"
        
        for index in range(len(author[index_pubmed])):
            select_author = f"SELECT Id_Author FROM AUTHORS WHERE Name = \"{author[index_pubmed][index]}\""
            Cursor.execute(select_author)
            myresult_author = Cursor.fetchall()
            id_author = None
            for res, in myresult_author:
                id_author = res


            if id_author is None:
                if index > len(affiliation_ids)-1:
                    if index == len(author[index_pubmed])-1:
                        sql += f" (\"{author[index_pubmed][index]}\", null)"
                    else:
                        sql += f" (\"{author[index_pubmed][index]}\", null),"
                else:
                    if index == len(author[index_pubmed])-1:
                        sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]})"
                    else:
                        sql += f" (\"{author[index_pubmed][index]}\", {affiliation_ids[index]}),"
    
        print(sql)
        if sql != "INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES":
                #remove ultimo caracter se este for uma ","
            if sql.endswith(","):
                sql = sql[:-1]

            Cursor.execute(sql)

    DataBase.commit()

    print(Cursor.rowcount, "record inserted.")

    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES ("Baranowsky W", 1)
INSERT INTO `Project_TJM`.AFFILIATION (Description) VALUES ("Mary Imogene Bassett Research Institute, Cooperstown, New York 13326.")
INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES ("Pearson TA", 2)
INSERT INTO `Project_TJM`.AFFILIATION (Description) VALUES ("Laboratoire de Biochimie Cellulaire, Facultes Universitaires Notre-Dame de la Paix, Bruxelles, Belgique.")
INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES ("Remacle J", 3), ("Arnould T", null), ("Michiels C", null)
INSERT INTO AUTHORS (Name, AFFILIATION_Id_affiliation ) VALUES ("Carninci P", 1), ("Kasukawa T", null), ("Katayama S", null), ("Gough J", null), ("Frith MC", null), ("Maeda N", null), ("Oyama R", null), ("Ravasi T", null), ("Lenhard B", null), ("Wells C", null), ("Kodzius R", null), ("Shimokawa K", null), ("Bajic VB", null), ("Brenner SE", null), ("Batalov S", null), ("Forrest AR", null), ("Zavolan M", null),

In [81]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="Project_TJM"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`PM_AUTHORS_relationship` (
        `PUBMED_Id_PM` INT NOT NULL,
        `AUTHORS_Id_Author` INT NOT NULL,
        PRIMARY KEY (`PUBMED_Id_PM`, `AUTHORS_Id_Author`)
    )
    """
    Cursor.execute(TableName)
    print(len(file_locus_pub_list))
    print(len(loc_name_list))

    for index_pm in range(len(pub_list)):

        if pub_list[index_pm] != '':
            select_pm_at = f"SELECT Id_PM FROM PUBMED WHERE PM_ID = \"{pub_list[index_pm]}\""
        
            Cursor.execute(select_pm_at)
            myresult_pm_at = Cursor.fetchall()
            id_pm_at = None
            for res, in myresult_pm_at:
                id_pm_at = res

            print(id_pm_at)
            author_id_list = []
            for xx in author[index_pm]:
                select_at_pm = f"SELECT Id_Author FROM AUTHORS WHERE Name= \"{xx}\""
                Cursor.execute(select_at_pm)
                myresult_at = Cursor.fetchall()
                id_at = None
                for res, in myresult_at:
                    id_at = res
                author_id_list.append(id_at)

            index = 0
            sql_at_pm_relation = "INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES" 
            author_id_list = list(dict.fromkeys(author_id_list))
            for id_at in author_id_list:
                if id_at is not None and id_pm_at is not None:      
                    #verificar se existe relação entre ids
                    select_rel_at_pm = f"SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = \"{id_pm_at}\" and AUTHORS_Id_Author = \"{id_at}\""
                    print(select_rel_at_pm)
                    Cursor.execute(select_rel_at_pm)
                    myresult_rel_pm_at = Cursor.fetchall()
                    id_rel_pm_at = None
                    for res_PUBMED_Id_PM,res_AUTHORS_Id_Author, in myresult_rel_pm_at:   #apenas selecionamos um dos id pq se não houver relação serão ambos null
                        id_rel_pm_at = res_PUBMED_Id_PM     

                    #se null:
                    if id_rel_pm_at is None:
                        if index == len(author_id_list)-1:
                            sql_at_pm_relation += f" ({id_pm_at}, {id_at})"
                        else:
                            sql_at_pm_relation += f" ({id_pm_at}, {id_at}),"
                index += 1

            print(sql_at_pm_relation)
            if sql_at_pm_relation != "INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES":
                #remove ultimo caracter se este for uma ","
                if sql_at_pm_relation.endswith(","):
                    sql_at_pm_relation = sql_at_pm_relation[:-1] #remove utimo caracter da string

                print(sql_at_pm_relation)
                Cursor.execute(sql_at_pm_relation)
                DataBase.commit()
                print(Cursor.rowcount, "record inserted.")

    
    
    
    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

6
6
2
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "2" and AUTHORS_Id_Author = "3"
INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES (2, 3)
INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES (2, 3)
1 record inserted.
3
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "3" and AUTHORS_Id_Author = "4"
INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES (3, 4)
INSERT INTO PM_AUTHORS_relationship (PUBMED_Id_PM, AUTHORS_Id_Author) VALUES (3, 4)
1 record inserted.
4
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "4" and AUTHORS_Id_Author = "5"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "4" and AUTHORS_Id_Author = "6"
SELECT PUBMED_Id_PM, AUTHORS_Id_Author FROM PM_AUTHORS_relationship WHERE PUBMED_Id_PM = "4" and AUTHORS_Id_Author = "7"
INSERT INTO

In [82]:
import mysql.connector as SQLC
try: 
    DataBase = SQLC.connect(
    host ="geo.di.uminho.pt",
    user ="bioinformatica",
    password ="20221207",
    database ="Project_TJM"
    )
    Cursor = DataBase.cursor()

    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`FEATURES` (
        `Id_F` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Gene` VARCHAR(45) NULL,
        `Source_location` VARCHAR(45) NULL
    
    )
    """
    Cursor.execute(TableName)

    TableName ="""CREATE TABLE IF NOT EXISTS `Project_TJM`.`CDS` (
        `Id_CDS` INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        `Translation_seq` longtext  NULL,
        `Location` VARCHAR(45)  NULL,
        `Product` VARCHAR(45)  NULL,
        `Protein_ID` VARCHAR(45) NULL
    )
    """
    Cursor.execute(TableName)

    for index_gb in range(len(loc_name_list)):

        #get genbank id
        select_gb = f"SELECT Id_GB FROM GENBANK WHERE GB_ID= \"{loc_name_list[index_gb]}\""
        Cursor.execute(select_gb)
        myresult_gb = Cursor.fetchall()
        id_gb = None
        for res, in myresult_gb:
            id_gb = res
        
        print(id_gb)
        if id_gb is not None:
            for index_ft in range(len(source[index_gb])):
                if source[index_gb][index_ft] != '':  #verificar se existe "source" na bd
                    sql_insert_source = "INSERT INTO FEATURES (Gene, Source_location) VALUES"
                    select_ft = f"SELECT Id_F FROM FEATURES WHERE Source_location = \"{source[index_gb][index_ft]}\""
            
                    Cursor.execute(select_ft)
                    myresult_ft = Cursor.fetchall()
                    id_ft = None
                    for res_ft, in myresult_ft:
                        id_ft = res_ft

                    if id_ft is None: #se n existe, insere
                        if len(gene[index_gb]) > 0:
                            sql_insert_source += f" (\"{gene[index_gb][index_ft]}\", \"{source[index_gb][index_ft]}\")"
                        else:
                            sql_insert_source += f" ( null, \"{source[index_gb][index_ft]}\")"

                        if sql_insert_source.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_source = sql_insert_source[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_source)

                        # print(Cursor.lastrowid)  #buscar o id do feature inserido
                        id_inserted_feature = Cursor.lastrowid
                    else:
                        id_inserted_feature = id_ft

                    #relationship between genbank and feature
                    sql_insert_gb_ft = "INSERT INTO GB_FT_relationship (GENBANK_Id_GB, FEATURES_Id_F) VALUES"
                    select_gb_ft = f"SELECT GENBANK_Id_GB FROM GB_FT_relationship WHERE GENBANK_Id_GB = {id_gb} and FEATURES_Id_F = {id_inserted_feature}"
            
                    Cursor.execute(select_gb_ft)
                    myresult_gb_ft = Cursor.fetchall()
                    id_gb_ft = None
                    for res_gb_ft, in myresult_gb_ft:
                        id_gb_ft = res_gb_ft

                    if id_gb_ft is None: #se n existe, insere
                        sql_insert_gb_ft += f" ({id_gb}, {id_inserted_feature})"

                        if sql_insert_gb_ft.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_gb_ft = sql_insert_gb_ft[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_gb_ft)

            for index_cds in range(len(CDS[index_gb])):
                print(CDS[index_gb])
                print(CDS[index_gb][index_cds])
                if CDS[index_gb][index_cds]:  #verificar se existe "cds" na bd
                    sql_insert_cds = "INSERT INTO CDS (Translation_seq, Location,Product,Protein_ID) VALUES"
                    select_cds = f"SELECT Id_CDS FROM CDS WHERE Translation_seq = \"{CDS[index_gb][index_cds][3]}\""
            
                    Cursor.execute(select_cds)
                    myresult_cds = Cursor.fetchall()
                    id_cds = None
                    for res_cds, in myresult_cds:
                        id_cds = res_cds

                    if id_cds is None: #se n existe, insere
                        sql_insert_cds += f" (\"{CDS[index_gb][index_cds][3]}\",\"{CDS[index_gb][index_cds][0]}\",\"{CDS[index_gb][index_cds][1]}\", \"{CDS[index_gb][index_cds][2]}\")"

                        if sql_insert_cds.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_cds = sql_insert_cds[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_cds)

                        # print(Cursor.lastrowid)  #buscar o id do cds inserido
                        id_inserted_cds = Cursor.lastrowid
                    else:
                        id_inserted_cds = id_cds
                    
                    #relationship between cds and genbank
                    sql_insert_gb_cds = "INSERT INTO CDS_GB_relationship (GENBANK_Id_GB, CDS_Id_CDS) VALUES"
                    select_gb_cds = f"SELECT GENBANK_Id_GB FROM CDS_GB_relationship WHERE GENBANK_Id_GB = {id_gb} and CDS_Id_CDS = {id_inserted_cds}"
            
                    Cursor.execute(select_gb_cds)
                    myresult_gb_cds = Cursor.fetchall()
                    id_gb_cds = None
                    for res_gb_cds, in myresult_gb_cds:
                        id_gb_ft = res_gb_cds

                    if id_gb_ft is None: #se n existe, insere
                        sql_insert_gb_cds += f" ({id_gb}, {id_inserted_cds})"

                        if sql_insert_gb_cds.endswith(","):  #remove ultimo caracter se este for uma ","
                            sql_insert_gb_cds = sql_insert_gb_cds[:-1] #remove utimo caracter da string
                        Cursor.execute(sql_insert_gb_cds)

                DataBase.commit()
                print(Cursor.rowcount, "record inserted.")

    
    
    
    Cursor.close()

except SQLC.Error as error:
    print("Failed to insert record into MySQL table {}".format(error))

9
[[], [], [], ['[67:889](+)', ['NADH dehydrogenase subunit'], ['CAA39694.1'], ['MRAALRLLATATATVRPSARFLKPGSPTGLTGLGTHPSPRSALLYLYNHTLDKLKQIPEHSLYRQSAEALTKHRLAIVEQYVPDGYDAWQERARKLLEKHKSDLTARQFDGQHARLVEGPDGRAYFIRQMVPPQDWRDVEWDGAVLDPHFSWVQTGEDVVGAVKLEDSDKLLELDKIRESDPVAYRQGLRDLGIKMGGVVEDKSPVEWESEPPLSAEQIAEMEARIGSGLIEEVVQVAEGELKLVDIMTQARPWEALEEEAPEGQWTYFERKE']], [], [], []]
[]
1 record inserted.
[[], [], [], ['[67:889](+)', ['NADH dehydrogenase subunit'], ['CAA39694.1'], ['MRAALRLLATATATVRPSARFLKPGSPTGLTGLGTHPSPRSALLYLYNHTLDKLKQIPEHSLYRQSAEALTKHRLAIVEQYVPDGYDAWQERARKLLEKHKSDLTARQFDGQHARLVEGPDGRAYFIRQMVPPQDWRDVEWDGAVLDPHFSWVQTGEDVVGAVKLEDSDKLLELDKIRESDPVAYRQGLRDLGIKMGGVVEDKSPVEWESEPPLSAEQIAEMEARIGSGLIEEVVQVAEGELKLVDIMTQARPWEALEEEAPEGQWTYFERKE']], [], [], []]
[]
1 record inserted.
[[], [], [], ['[67:889](+)', ['NADH dehydrogenase subunit'], ['CAA39694.1'], ['MRAALRLLATATATVRPSARFLKPGSPTGLTGLGTHPSPRSALLYLYNHTLDKLKQIPEHSLYRQSAEALTKHRLAIVEQYVPDGYDAWQERARKLLEKHKSDLTARQFDGQHARLVEGPDGRAYFIRQMVPPQDWR